# Lab 2.6.3 Solution

This solution notebook contains completed exercises from Lab 2.6.3.

**Note:** The main lab notebooks contain fully working code throughout. This solution notebook focuses on the "Try It Yourself" exercises and challenges.

---

## How to Use This Solution

1. First attempt the exercises in the main lab notebook
2. If stuck, check the hints in the main notebook
3. Compare your solution with this reference
4. Understand the differences and learn from them

---

## Exercise Solutions

### Challenge 1: Architecture Control

```python
# Solution: Draw a building outline and generate in 5 architectural styles

import numpy as np
import cv2
from PIL import Image

def draw_building_outline(size=(1024, 1024)):
    """Draw a simple building outline for ControlNet."""
    img = np.zeros((*size, 3), dtype=np.uint8)
    
    # Main building body
    cv2.rectangle(img, (300, 400), (700, 900), (255, 255, 255), 2)
    
    # Roof (triangular)
    pts = np.array([[250, 400], [500, 150], [750, 400]], np.int32)
    cv2.polylines(img, [pts], True, (255, 255, 255), 2)
    
    # Windows
    for row in range(2):
        for col in range(3):
            x = 350 + col * 120
            y = 480 + row * 180
            cv2.rectangle(img, (x, y), (x + 80, y + 120), (255, 255, 255), 2)
    
    # Door
    cv2.rectangle(img, (450, 700), (550, 900), (255, 255, 255), 2)
    
    return Image.fromarray(img)

# Create the building sketch
building_sketch = draw_building_outline()

# Generate in 5 architectural styles
architectural_styles = [
    "A modern minimalist house, clean lines, glass and concrete, architectural photography",
    "A Victorian Gothic mansion, ornate details, dark atmosphere, dramatic",
    "A Mediterranean villa, terracotta roof, warm colors, sunny day",
    "A Japanese traditional house, wooden architecture, zen garden, peaceful",
    "A futuristic eco-house, living walls, solar panels, sustainable architecture",
]

for i, prompt in enumerate(architectural_styles):
    generator = torch.Generator(device="cuda").manual_seed(42)
    image = pipe_canny(
        prompt=prompt,
        image=building_sketch,
        controlnet_conditioning_scale=0.7,
        num_inference_steps=25,
        generator=generator,
    ).images[0]
    image.save(f"architecture_style_{i}.png")
```

### Challenge 2: Multi-ControlNet (Edges + Depth)

```python
# Solution: Combine edges and depth for enhanced control

from diffusers import ControlNetModel, StableDiffusionXLControlNetPipeline

# Load multiple ControlNets
controlnet_canny = ControlNetModel.from_pretrained(
    "diffusers/controlnet-canny-sdxl-1.0", torch_dtype=torch.bfloat16
)
controlnet_depth = ControlNetModel.from_pretrained(
    "diffusers/controlnet-depth-sdxl-1.0", torch_dtype=torch.bfloat16
)

# Create multi-ControlNet pipeline
pipe_multi = StableDiffusionXLControlNetPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    controlnet=[controlnet_canny, controlnet_depth],
    torch_dtype=torch.bfloat16,
)
pipe_multi = pipe_multi.to("cuda")

# Generate with both controls
# Note: controlnet_conditioning_scale takes a list for multiple controls
image = pipe_multi(
    prompt="A fantasy castle, magical atmosphere, detailed",
    image=[canny_edges, depth_map],  # Both control images
    controlnet_conditioning_scale=[0.5, 0.5],  # Balance both
    num_inference_steps=25,
).images[0]
```

## Tips from This Lab

### Best Practices Learned:

1. Always set a random seed for reproducible results
2. Use negative prompts to avoid common artifacts
3. Start with default parameters, then tune
4. Save metadata with every generation
5. Use bfloat16 on DGX Spark for optimal performance